In [1]:
server_args = []
# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

from manipulation import running_as_notebook

# Imports
import numpy as np
import pydot
from ipywidgets import Dropdown, Layout
from IPython.display import display, HTML, SVG

from pydrake.all import (
    AddMultibodyPlantSceneGraph, ConnectMeshcatVisualizer, DiagramBuilder, 
    FindResourceOrThrow, GenerateHtml, InverseDynamicsController, 
    MultibodyPlant, Parser, Simulator)
from pydrake.multibody.jupyter_widgets import MakeJointSlidersThatPublishOnCallback
import pydrake
from pydrake import geometry
from pydrake.math import RigidTransform, RollPitchYaw, RotationMatrix 
from pydrake.solvers.mathematicalprogram import MathematicalProgram, Solve
from pydrake.systems.jupyter_widgets import PoseSliders, WidgetSystem
from ipywidgets import ToggleButton, ToggleButtons
from functools import partial
from pydrake.all import (
    JointIndex, PiecewisePolynomial, JacobianWrtVariable,
    eq, ge,  AutoDiffXd, autoDiffToGradientMatrix, SnoptSolver, IpoptSolver,
    initializeAutoDiffGivenGradientMatrix, autoDiffToValueMatrix, autoDiffToGradientMatrix,
    AddUnitQuaternionConstraintOnPlant, PositionConstraint, OrientationConstraint,
    MeshcatContactVisualizer, Expression
)

import pickle
from pydrake.symbolic import Variable

In [2]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)

In [3]:
#add table
ground = pydrake.multibody.parsing.Parser(plant, scene_graph).AddModelFromFile(
            pydrake.common.FindResourceOrThrow(
                "drake/examples/push_box/ground.urdf"), "ground")

#add box
cracker_box = pydrake.multibody.parsing.Parser(plant, scene_graph).AddModelFromFile(
            pydrake.common.FindResourceOrThrow(
                "drake/examples/push_box/003_cracker_box_plannar_test.urdf"), "cracker_box")

X_WB = RigidTransform(RotationMatrix.MakeXRotation(np.pi/2), [0., 0., 0.104200 + 1e-07])#
planar_joint_frame = plant.AddFrame(pydrake.multibody.tree.FixedOffsetFrame("planar_joint_frame", plant.world_frame(), X_WB))
box_joint = plant.AddJoint(pydrake.multibody.tree.PlanarJoint("box_joint", planar_joint_frame, plant.GetFrameByName("base_link_cracker", cracker_box)))
box_joint.set_default_translation([0, 0])

In [4]:
plant.Finalize()
plant.set_name("plant")

In [5]:
cracker_box_body = plant.GetBodyByName("base_link_cracker", cracker_box)
cracker_box_id = plant.GetBodyFrameIdIfExists(cracker_box_body.index())


draw_frames = True
frames_to_draw = [cracker_box_id] if draw_frames else []
# meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, frames_to_draw=frames_to_draw, axis_length=0.15,
#                  axis_radius=0.001,)
# # meshcat.set_planar_viewpoint(xmin=-.5, xmax=.5, ymin=-0.1, ymax=0.6)

# #visualizer the contact
# contact_visualizer = builder.AddSystem(MeshcatContactVisualizer(meshcat, 
#                                        force_threshold=0.0,
#                                        contact_force_scale=0.05, 
#                                        contact_force_radius=0.001,   
#                                        plant=plant))
# builder.Connect(plant.get_contact_results_output_port(),
#                 contact_visualizer.GetInputPort("contact_results"))


diagram = builder.Build()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
# X_WB = RigidTransform(RotationMatrix(), [0., 0., 0.0334 + 1e-07])
# plant.SetFreeBodyPose(plant_context, 
#                               plant.GetBodyByName("base_link_cracker", cracker_box),
#                               X_WB)

# meshcat.load()
diagram.Publish(context)

In [6]:
# query_object = plant.get_geometry_query_input_port().Eval(plant_context)
# scene_graph_inspector = query_object.inspector()
# collision_candidate_pairs = scene_graph_inspector.GetCollisionCandidates()
# signed_distance_pairs = query_object.ComputeSignedDistancePairwiseClosestPoints()
# for signed_distance_pair in signed_distance_pairs:
#     print("id_A: ", signed_distance_pair.id_A, "   id_B: ", signed_distance_pair.id_B)
#     print("p_ACa: ", signed_distance_pair.p_ACa, "   p_BCb: ", signed_distance_pair.p_BCb)
#     print("nhat_BA_W: ", signed_distance_pair.nhat_BA_W, "   distance: ", signed_distance_pair.distance)
#     print("########################")

In [7]:
a = np.array([[0, 0],[0, 0]])
print(a.shape)

(2, 2)


In [8]:
init_from_file = False
# init_from_file = True
# tmpfolder = 'tmp/'
# with open(tmpfolder +  'test_only_box_passive_scenegraph_sliding_friction/sol.pkl', 'rb' ) as file:
#     q_o_ini, v_o_ini, vdot_o_ini, f_list_ini, alpha_beta_list_ini, f_static_list_ini, f_sliding_list_ini, sliding_c_list_ini = pickle.load( file )

In [9]:
# Need this because a==b returns True even if a = AutoDiffXd(1, [1, 2]), b= AutoDiffXd(2, [3, 4])
# That's the behavior of AutoDiffXd in C++, also.
def autoDiffArrayEqual(a,b):
    return np.array_equal(a, b) and np.array_equal(autoDiffToGradientMatrix(a), autoDiffToGradientMatrix(b))


N = 10


# Create one context per timestep (to maximize cache hits)
#to use scenegraph, so the plant context need derive from diagram
context_list = [diagram.CreateDefaultContext() for i in range(N)]
plant_context_list = [plant.GetMyContextFromRoot(context_list[i]) for i in range(N)]
# We could get rid of this by implementing a few more Jacobians in MultibodyPlant:
ad_diagram = diagram.ToAutoDiffXd()
ad_plant = ad_diagram.GetSubsystemByName("plant")

# Make a new autodiff context (to maximize cache hits)
#to use scenegraph, so the plant context need derive from diagram
ad_context_list = [ad_diagram.CreateDefaultContext() for i in range(N)]
ad_plant_context_list = [ad_plant.GetMyContextFromRoot(ad_context_list[i]) for i in range(N)]

mu = 1
total_mass = sum(plant.get_body(index).get_mass(context) for index in plant.GetBodyIndices(cracker_box))
gravity = plant.gravity_field().gravity_vector()
g = -gravity[2]
m = total_mass



#time step constrain
T = 1
prog = MathematicalProgram()
# Time steps    
h = prog.NewContinuousVariables(N-1, "h")
prog.AddBoundingBoxConstraint([0.5*T/(N-1)]*(N-1), [2.0*T/(N-1)]*(N-1), h) 
prog.AddLinearConstraint(sum(h) >= .9*T)
prog.AddLinearConstraint(sum(h) <= 1.1*T)
prog.SetInitialGuess(h, [T/(N-1)]*(N-1))

#q, v, vdot constrain
nq_o = plant.num_positions(cracker_box)
nv_o = plant.num_velocities(cracker_box)
q_o = prog.NewContinuousVariables(nq_o, N, "q_o")
v_o = prog.NewContinuousVariables(nv_o, N, "v_o")
vdot_o = prog.NewContinuousVariables(nv_o, N-1, "vdot_o")

#initial constrain
q0_o = plant.GetPositions(plant_context, cracker_box)
v0_o = plant.GetVelocities(plant_context, cracker_box)
vdot0_o = v0_o
v0_o[0] = 0.5
# vdot0_o[3] = -9.81
prog.AddBoundingBoxConstraint(q0_o, q0_o, q_o[:,0])
prog.AddBoundingBoxConstraint(v0_o, v0_o, v_o[:,0])
for n in range(N):
#     prog.AddQuadraticErrorCost(np.diag([1]*nq_o), q0_o, q_o[:,n])
    prog.AddQuadraticErrorCost(np.diag([100]*nv_o), [0]*nv_o, v_o[:,n])
    if not init_from_file:
        prog.SetInitialGuess(q_o[:,n], q0_o)
#         if n < 10:
#             prog.SetInitialGuess(vdot_o[:,n], vdot0_o)
#         if n < 50:
#             q0_o[0] =  n*0.001
#             prog.SetInitialGuess(q_o[:,n], q0_o)
#             v0_o[0] = 0.5 - n*0.01
#             prog.SetInitialGuess(v_o[:,n], v0_o)
#         else:
#             q0_o[0] =  50*0.001
#             prog.SetInitialGuess(q_o[:,n], q0_o)
#             v0_o[0] = 0
#             prog.SetInitialGuess(v_o[:,n], v0_o)
    # Unit quaternions
#     AddUnitQuaternionConstraintOnPlant(plant, q_o[:,n], prog)

    
if init_from_file:
    prog.SetInitialGuess(q_o, q_o_ini)
    prog.SetInitialGuess(v_o, v_o_ini)
    prog.SetInitialGuess(vdot_o, vdot_o_ini)

In [10]:
# def velocity_position_constraint(vars, context_index):
#     h, q, v, qn = np.split(vars, [1, 1+nq_o, 1+nq_o+nv_o])
#     if isinstance(vars[0], AutoDiffXd):
#         if not autoDiffArrayEqual(q, ad_plant.GetPositions(ad_plant_context_list[context_index], cracker_box)):
#             ad_plant.SetPositions(ad_plant_context_list[context_index], q)
#         v_from_qdot = ad_plant.MapQDotToVelocity(ad_plant_context_list[context_index], (qn - q)/h)
#     else:
#         if not np.array_equal(q, plant.GetPositions(plant_context_list[context_index])):
#             plant.SetPositions(plant_context_list[context_index], q)
#         v_from_qdot = plant.MapQDotToVelocity(plant_context_list[context_index], (qn - q)/h)
#     return v - v_from_qdot
# for n in range(N-1):
#     prog.AddConstraint(
#         partial(velocity_position_constraint, context_index=n), 
#         lb=[0]*nv_o, ub=[0]*nv_o, 
#         vars=np.concatenate(([h[n]], q_o[:,n], v_o[:,n+1], q_o[:,n+1])))  #(implicit Euler)
    
#accelaration and velocity relation constrain(implicit Euler)
for n in range(N-1):
    q_v_constrain = prog.AddConstraint(eq(q_o[:, n+1], q_o[:,n] + h[n]*v_o[:,n+1]))#(implicit Euler)
    v_vdot_constrain = prog.AddConstraint(eq(v_o[:, n+1], v_o[:,n] + h[n]*vdot_o[:,n]))#(implicit Euler)
    q_v_constrain.evaluator().set_description(f"q_v_constrain_{n}")
    v_vdot_constrain.evaluator().set_description(f"v_vdot_constrain_{n}")

In [11]:
query_object = plant.get_geometry_query_input_port().Eval(plant_context)
scene_graph_inspector = query_object.inspector()
collision_candidate_pairs = scene_graph_inspector.GetCollisionCandidates()

#set the ignored_collision_pairs
cracker_box_body = plant.GetBodyByName("base_link_cracker", cracker_box)
cracker_box_id = plant.GetBodyFrameIdIfExists(cracker_box_body.index())
ground_body = plant.GetBodyByName("ground", ground)
ground_id = plant.GetBodyFrameIdIfExists(ground_body.index())
ground_geometry_id = scene_graph_inspector.GetGeometryIdByName(ground_id, pydrake.geometry.Role.kProximity, "ground::ground_collision")
box_collision_geometry_id = scene_graph_inspector.GetGeometryIdByName(cracker_box_id, pydrake.geometry.Role.kProximity, "cracker_box::box_collision")
point_collision1_geometry_id = scene_graph_inspector.GetGeometryIdByName(cracker_box_id, pydrake.geometry.Role.kProximity, "cracker_box::point_collision1")
point_collision3_geometry_id = scene_graph_inspector.GetGeometryIdByName(cracker_box_id, pydrake.geometry.Role.kProximity, "cracker_box::point_collision3")

ignored_collision_pairs = set([(ground_geometry_id, box_collision_geometry_id)
#                                ,(ground_geometry_id, point_collision1_geometry_id)
#                                ,(ground_geometry_id, point_collision3_geometry_id)

                              ])
#it's the top four collision point
collision_pairs1 = set([
                               (ground_geometry_id, point_collision1_geometry_id)
                               ,(ground_geometry_id, point_collision3_geometry_id)
                              ])

#set collision_candidate_pair_list
collision_candidate_pair_list = []
for collision_candidate_pair in collision_candidate_pairs:
    if collision_candidate_pair not in ignored_collision_pairs:
        collision_candidate_pair_list.append(collision_candidate_pair)

        
N_f = len(collision_candidate_pair_list)        

#express in contact coordinate
lambd_z = prog.NewContinuousVariables(N_f, N-1, "lambd_z")
lambd_x_pos = prog.NewContinuousVariables(N_f, N-1, "lambd_x_pos")
lambd_x_neg = prog.NewContinuousVariables(N_f, N-1, "lambd_x_neg")
gama = prog.NewContinuousVariables(N_f, N-1, "gama")
#eq9
for n in range(N-1):
    prog.AddBoundingBoxConstraint([0]*N_f, [np.inf]*N_f, lambd_z[:, n])
    prog.AddBoundingBoxConstraint([0]*N_f, [np.inf]*N_f, lambd_x_pos[:, n])
    prog.AddBoundingBoxConstraint([0]*N_f, [np.inf]*N_f, lambd_x_neg[:, n])
    prog.AddBoundingBoxConstraint([0]*N_f, [np.inf]*N_f, gama[:, n])
    
    #eq10
    eq10_constraint = prog.AddConstraint(ge(mu*lambd_z[:, n] - lambd_x_pos[:, n] - lambd_x_neg[:, n], 0))
    eq10_constraint.evaluator().set_description(f"eq10_constraint_{n}")
#     for k in range(N_f):
#         if collision_candidate_pair_list[k] not in collision_pairs1:
#             prog.SetInitialGuess(lambd_z[k, n], m*g/2)

In [12]:
iter_count = 0
#reference drake sliding_friction_complementarity_constraint.cc file
def friction_complementarity_constraint(vars, context_index):
    q, v, lambd_z, lambd_x_pos, lambd_x_neg, gama = np.split(vars, 
                                        [nq_o, nq_o+nv_o, nq_o+nv_o+N_f, nq_o+nv_o+N_f*2, 
                                          nq_o+nv_o+N_f*3])

    #to deal the error,"PyFunctionConstraint: Output must be of scalar type float. Got AutoDiffXd instead."
    #need eval_plant and eval_context for both AutoDiffXd and scalar type input
    if isinstance(vars[0], AutoDiffXd):
        #set plant pos and vel
        if not autoDiffArrayEqual(q, ad_plant.GetPositions(ad_plant_context_list[context_index], cracker_box)):
            ad_plant.SetPositions(ad_plant_context_list[context_index], cracker_box, q)
        if not autoDiffArrayEqual(v, ad_plant.GetVelocities(ad_plant_context_list[context_index], cracker_box)):
            ad_plant.SetVelocities(ad_plant_context_list[context_index], cracker_box, v)
        
        eval_plant = ad_plant
        eval_plant_context = ad_plant_context_list[context_index]
    else:
        if not np.array_equal(q, plant.GetPositions(plant_context_list[context_index], cracker_box)):
            plant.SetPositions(plant_context_list[context_index], cracker_box, q)
        if not np.array_equal(v, plant.GetVelocities(plant_context_list[context_index], cracker_box)):
            plant.SetVelocities(plant_context_list[context_index], cracker_box, v)
        
        eval_plant = plant
        eval_plant_context = plant_context_list[context_index]
    
    #get query object and scenegraph inspector
    eval_query_object = eval_plant.get_geometry_query_input_port().Eval(eval_plant_context)
    eval_scene_graph_inspector = eval_query_object.inspector()
    
    global iter_count
    if iter_count % (100*N) == 0:
        print("iter_count:",iter_count/N)
    iter_count = iter_count + 1
    res_list = []
    #f is A exert to B express in W
    for n in range(N_f):
        signed_distance_pair = eval_query_object.ComputeSignedDistancePairClosestPoints(
                                    collision_candidate_pair_list[n][0], collision_candidate_pair_list[n][1])
        #derive friction coefficience
        geometryA_props = eval_scene_graph_inspector.GetProximityProperties(signed_distance_pair.id_A)
        geometryB_props = eval_scene_graph_inspector.GetProximityProperties(signed_distance_pair.id_B)
        geometryA_friction = geometryA_props.GetProperty("material", "coulomb_friction")
        geometryB_friction = geometryB_props.GetProperty("material", "coulomb_friction")
        combined_friction = pydrake.multibody.plant.CalcContactFrictionFromSurfaceProperties(
                                                        geometryA_friction, geometryB_friction)
        
        #calc the velocity of contact point Cb of geometry B  related to frame A express in world frame
        frame_A_id = eval_scene_graph_inspector.GetFrameId(signed_distance_pair.id_A)
        frame_B_id = eval_scene_graph_inspector.GetFrameId(signed_distance_pair.id_B)
        frameA = eval_plant.GetBodyFromFrameId(frame_A_id).body_frame()
        frameB = eval_plant.GetBodyFromFrameId(frame_B_id).body_frame()
        
        V_AB_W = frameB.CalcSpatialVelocity(eval_plant_context, frameA, eval_plant.world_frame())
        
        X_BGb = eval_scene_graph_inspector.GetPoseInFrame(signed_distance_pair.id_B)
        p_GbCb = signed_distance_pair.p_BCb
        if isinstance(vars[0], AutoDiffXd):
            p_BCb = (X_BGb.cast[AutoDiffXd]()).multiply(p_GbCb.reshape(3, 1))
        else:
            p_BCb = X_BGb.multiply(p_GbCb.reshape(3, 1))
        p_BCb_W = (eval_plant.CalcRelativeTransform(eval_plant_context, 
                                                 eval_plant.world_frame(), frameB).rotation()).multiply(p_BCb.reshape(3, 1))
        v_ACb_W = V_AB_W.Shift(p_BCb_W).translational()
        
        
        nhat_BA_W = np.array([signed_distance_pair.nhat_BA_W]).T
        
        v_sliding_ACb_W = (np.identity(3) - nhat_BA_W * nhat_BA_W.T).dot(v_ACb_W)
        v_sliding_ACb_W_2d = np.array([v_sliding_ACb_W[0], v_sliding_ACb_W[2]])
        tranform_2d = -np.array([[nhat_BA_W[2], -nhat_BA_W[0]],[nhat_BA_W[0], nhat_BA_W[2]]]).reshape(2, 2)
        thi_2d = tranform_2d.dot(v_sliding_ACb_W_2d)
        thi = thi_2d[0]

        phi = signed_distance_pair.distance
        
        lambd_z, lambd_x_pos, lambd_x_neg, gama
        #eq8,11-16
        res = np.concatenate((
        [phi],
        [phi*lambd_z[n]],
        [gama[n] + thi],
        [gama[n] - thi],
        [gama[n]*(mu*lambd_z[n] - lambd_x_pos[n] - lambd_x_neg[n])],
        [(gama[n] + thi)*lambd_x_pos[n]],
        [(gama[n] - thi)*lambd_x_neg[n]]
        )) #7
        
        
        res_list.append(res)
    if iter_count % (100*N) == 0:
        print("res_list:", autoDiffToValueMatrix(res_list)
             )
    #return must be 1 dimension vector, like [r1, r2, ...]
    res_list = np.array(res_list).reshape(7*N_f,)
    return res_list

epsilon = 1e-8
epsilon1 = epsilon
epsilon2 = epsilon
epsilon3 = epsilon
epsilon4 = epsilon
for n in range(N-1):
    #set vars be 1 dimension vector, like [r1, r2, ...]
    vars = np.concatenate((q_o[:, n+1], v_o[:, n+1], 
                            lambd_z[:, n], 
                            lambd_x_pos[:, n],
                            lambd_x_neg[:, n],
                            gama[:, n],
                          ))
    friction_constraint = prog.AddConstraint(
        partial(friction_complementarity_constraint, context_index=n), 
        lb=np.array([np.concatenate(([0],          [0],     [0],       [0],       [0],         [0],         [0])) for i in range(N_f)]).reshape(7*N_f,), 
        ub=np.array([np.concatenate(([np.inf], [epsilon1], [np.inf], [np.inf],  [epsilon2],  [epsilon3], [epsilon4])) for i in range(N_f)]).reshape(7*N_f,), 
        vars=vars)   
    friction_constraint.evaluator().set_description(f"friction_constraint_{n}")

In [13]:
iter_count1 = 0
def manipulator_equations(vars, context_index):
    q, v, vdot, lambd_z, lambd_x_pos, lambd_x_neg= np.split(vars, [nq_o, nq_o+nv_o, nq_o+nv_o*2, nq_o+nv_o*2+N_f, nq_o+nv_o*2+N_f*2])
    #f and alpha_beta contain all collsion candidates force and corresponding alpha,beta
    lambd_x = lambd_x_pos - lambd_x_neg
    f = np.vstack((np.array([lambd_x]), np.array([lambd_z]))).T
    
    
    #to deal the error,"PyFunctionConstraint: Output must be of scalar type float. Got AutoDiffXd instead."
    #need eval_plant and eval_context for both AutoDiffXd and scalar type input
    if isinstance(vars[0], AutoDiffXd):
        #set plant pos and vel
        if not autoDiffArrayEqual(q, ad_plant.GetPositions(ad_plant_context_list[context_index], cracker_box)):
            ad_plant.SetPositions(ad_plant_context_list[context_index], cracker_box, q)
        if not autoDiffArrayEqual(v, ad_plant.GetVelocities(ad_plant_context_list[context_index], cracker_box)):
            ad_plant.SetVelocities(ad_plant_context_list[context_index], cracker_box, v)
        
        eval_plant = ad_plant
        eval_plant_context = ad_plant_context_list[context_index]
    else:
        if not np.array_equal(q, plant.GetPositions(plant_context_list[context_index], cracker_box)):
            plant.SetPositions(plant_context_list[context_index], cracker_box, q)
        if not np.array_equal(v, plant.GetVelocities(plant_context_list[context_index], cracker_box)):
            plant.SetVelocities(plant_context_list[context_index], cracker_box, v)
        
        eval_plant = plant
        eval_plant_context = plant_context_list[context_index]
    
    
    # matrices for the manipulator equations
    M = eval_plant.CalcMassMatrixViaInverseDynamics(eval_plant_context)
    Cv = eval_plant.CalcBiasTerm(eval_plant_context)
    tauG = eval_plant.CalcGravityGeneralizedForces(eval_plant_context)

    
    #tauF = sumi(J.T*F), use object query and scenegraph inspector to calc J
    tauF = np.zeros((nv_o))
    #get query object and scenegraph inspector
    eval_query_object = eval_plant.get_geometry_query_input_port().Eval(eval_plant_context)
    eval_scene_graph_inspector = eval_query_object.inspector()
    
    global iter_count1
    iter_count1 = iter_count1 + 1
    #f is A exert to B express in W
    for n in range(N_f):
        signed_distance_pair = eval_query_object.ComputeSignedDistancePairClosestPoints(
                                    collision_candidate_pair_list[n][0], collision_candidate_pair_list[n][1])
        
        frame_A_id = eval_scene_graph_inspector.GetFrameId(signed_distance_pair.id_A)
        frame_B_id = eval_scene_graph_inspector.GetFrameId(signed_distance_pair.id_B)
        frameA = eval_plant.GetBodyFromFrameId(frame_A_id).body_frame()
        frameB = eval_plant.GetBodyFromFrameId(frame_B_id).body_frame()
        
        X_AGa = eval_scene_graph_inspector.GetPoseInFrame(signed_distance_pair.id_A)
        p_GaCa = signed_distance_pair.p_ACa
        if isinstance(vars[0], AutoDiffXd):
            p_ACa = (X_AGa.cast[AutoDiffXd]()).multiply(p_GaCa.reshape(3, 1))
        else:
            p_ACa = X_AGa.multiply(p_GaCa.reshape(3, 1))
            
        X_BGb = eval_scene_graph_inspector.GetPoseInFrame(signed_distance_pair.id_B)
        p_GbCb = signed_distance_pair.p_BCb
        if isinstance(vars[0], AutoDiffXd):
            p_BCb = (X_BGb.cast[AutoDiffXd]()).multiply(p_GbCb.reshape(3, 1))
        else:
            p_BCb = X_BGb.multiply(p_GbCb.reshape(3, 1))
        
        
        Jv_v_WCa = eval_plant.CalcJacobianTranslationalVelocity(eval_plant_context, JacobianWrtVariable.kV,
                                                    frameA, p_ACa,
                                                    eval_plant.world_frame(), eval_plant.world_frame())
        Jv_v_WCb = eval_plant.CalcJacobianTranslationalVelocity(eval_plant_context, JacobianWrtVariable.kV,
                                                    frameB, p_BCb,
                                                    eval_plant.world_frame(), eval_plant.world_frame())
        #discard y component in 2D
        Jv_v_WCa = Jv_v_WCa[[0, 2]]
        Jv_v_WCb = Jv_v_WCb[[0, 2]]
        
        nhat_BA_W = np.array([signed_distance_pair.nhat_BA_W]).T
        tranform_2d = -np.array([[nhat_BA_W[2], nhat_BA_W[0]],[-nhat_BA_W[0], nhat_BA_W[2]]]).reshape(2, 2)
        #transform f to world coordinate
        f_W = tranform_2d.dot(f[n])
        
        tauF = tauF + Jv_v_WCa.T.dot(-f_W) +  Jv_v_WCb.T.dot(f_W)

    global iter_count
    if iter_count1 % (100*N) == 0:
        print("manipulator_equations:", autoDiffToValueMatrix(M.dot(vdot) + Cv - tauG - tauF)
           ,"     tauF:", autoDiffToValueMatrix(tauF)
             )
    #return must be 1 dimension vector, like [r1, r2, ...]
    return M.dot(vdot) + Cv - tauG - tauF


# manipulator equations for all t (implicit Euler)
for n in range(N-1):
    vars = np.concatenate((q_o[:, n+1], v_o[:, n+1], vdot_o[:, n], 
                            lambd_z[:, n], 
                            lambd_x_pos[:, n],
                            lambd_x_neg[:, n]))
    manipulator_equations_constraint = prog.AddConstraint(partial(manipulator_equations, context_index=n), lb=[0]*nv_o, ub=[0]*nv_o, vars=vars)
    manipulator_equations_constraint.evaluator().set_description(f"manipulator_equations_constraint_{n}")

In [14]:
snopt = SnoptSolver().solver_id()
prog.SetSolverOption(snopt, 'Iterations Limits', 1e5 if running_as_notebook else 1)
prog.SetSolverOption(snopt, 'Major Iterations Limit', 200 if running_as_notebook else 1)
prog.SetSolverOption(snopt, 'Major Feasibility Tolerance', 5e-6)
prog.SetSolverOption(snopt, 'Major Optimality Tolerance', 1e-4)
prog.SetSolverOption(snopt, 'Superbasics limit', 2000)
prog.SetSolverOption(snopt, 'Linesearch tolerance', 0.9)
prog.SetSolverOption(snopt, 'Print file', 'snopt.out')


# ipopt = IpoptSolver()
# result = ipopt.Solve(prog)

# sno = SnoptSolver()
# result = sno.Solve(prog)

result = Solve(prog)
print(result.get_solver_id().name())
if not result.is_success():
#     print(result.GetInfeasibleConstraintNames(prog))
    print("solve failed!")
else:
    print("solve success")
    

iter_count: 0.0
SNOPT/fortran
solve success


In [15]:
print(result.GetInfeasibleConstraintNames(prog))

[]


In [16]:
h_sol = result.GetSolution(h)
print("h:", h_sol)

h: [0.11682721 0.13904947 0.06152822 0.05555556 0.18310014 0.17567402
 0.05555556 0.05555556 0.05715427]


In [17]:
q_o_sol = result.GetSolution(q_o)
print("q_o_sol:", q_o_sol)

q_o_sol: [[ 0.00000000e+00 -6.65013777e-18  1.78406529e-07  3.92802259e-14
   3.93146714e-14  4.03879603e-14  5.66405289e-14  5.66405289e-14
   5.66405289e-14  5.66405326e-14]
 [ 0.00000000e+00 -7.70743980e-18 -1.36116121e-07 -8.22887214e-18
  -3.47232252e-15 -2.87563601e-18  1.10022836e-17 -2.87557105e-18
   6.19310680e-16  8.87663208e-16]
 [ 0.00000000e+00  2.62406483e-17 -1.71215277e-06  4.38154050e-14
   4.38154055e-14 -8.18081626e-22 -8.40415502e-22  8.74032639e-25
   2.17726092e-15  2.72075585e-17]]


In [18]:
v_o_sol = result.GetSolution(v_o)
print("v_o_sol:", v_o_sol)

v_o_sol: [[ 5.00000000e-01 -3.83013477e-14  1.28298379e-06 -2.89663615e-06
   6.20018841e-16  1.90945217e-14 -9.22985456e-14  3.73846863e-24
  -4.08373727e-15  6.51057220e-20]
 [ 0.00000000e+00 -1.22699909e-14 -9.78842712e-07  2.21000342e-06
   1.74279994e-16  7.52900481e-12  3.25545702e-13 -2.16168724e-16
   1.11993527e-14  4.83039173e-15]
 [ 0.00000000e+00  3.67253328e-13 -1.23126898e-05  2.77987848e-05
   6.37841655e-21 -1.83348481e-13  8.85781737e-13  1.55353619e-15
   3.91907101e-14 -3.86999460e-14]]


In [19]:
vdot_o_sol = result.GetSolution(vdot_o)
print("vdot_o_sol:", vdot_o_sol)

vdot_o_sol: [[-4.27982489e+00  9.22704380e-06 -6.78650116e-05  5.21394507e-05
  -3.37731318e-09 -5.37013277e-11  1.65787241e-12 -7.00058501e-14
   6.36558214e-07]
 [-5.42889020e-14 -7.03949957e-06  5.17749517e-05 -3.97800616e-05
   2.84309470e-09  2.39924753e-11 -5.86371368e-12  2.05479188e-13
   4.45609017e-07]
 [ 2.32055642e-11 -8.85448459e-05  6.51295028e-04 -5.00378126e-04
   3.21679628e-08 -2.02282358e-13 -1.59161076e-11  6.77470233e-13
  -6.10899811e-06]]


In [20]:
lambd_z_sol = result.GetSolution(lambd_z)
print("lambd_z_sol:", lambd_z_sol)
print("\n")
print("lambd_z_sol:", lambd_z_sol[2] + lambd_z_sol[3])

lambd_z_sol: [[4.79833930e-08 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 4.79848224e-08 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [2.10023219e-01 4.90500534e-01 4.90487868e-01 4.90509236e-01
  4.90499999e-01 4.90500000e-01 4.90500000e-01 4.90500000e-01
  4.90500159e-01]
 [7.70976733e-01 4.90498762e-01 4.90517262e-01 4.90486786e-01
  4.90500001e-01 4.90500000e-01 4.90500000e-01 4.90500000e-01
  4.90499885e-01]]


lambd_z_sol: [0.98099995 0.9809993  0.98100513 0.98099602 0.981      0.981
 0.981      0.981      0.98100004]


In [21]:
lambd_x_pos_sol = result.GetSolution(lambd_x_pos)
lambd_x_neg_sol = result.GetSolution(lambd_x_neg)
lambd_x_sol = lambd_x_pos_sol - lambd_x_neg_sol
print("lambd_x_sol:", lambd_x_sol)

lambd_x_sol: [[-4.79833930e-08  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  4.79848224e-08  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [-4.03111964e-05  9.22704380e-07 -6.83448599e-06  0.00000000e+00
  -3.37731318e-10  0.00000000e+00  1.65787241e-13 -7.00047659e-15
   0.00000000e+00]
 [-4.27942130e-01  0.00000000e+00  0.00000000e+00  5.21394507e-06
   0.00000000e+00 -5.37013277e-12  0.00000000e+00  0.00000000e+00
   6.36558214e-08]]


In [22]:
gama_sol = result.GetSolution(gama)
print("gama_sol:", gama_sol)

gama_sol: [[ 7.65691078e-14  2.56596456e-06  5.79326674e-06 -2.15405552e-15
  -3.81994152e-14  1.84596914e-13  5.49299626e-25 -8.16740339e-15
  -1.22471466e-19]
 [ 7.65691078e-14  2.56596512e-06  5.79326674e-06 -9.14021312e-16
  -3.81994152e-14  1.84596914e-13  5.49299626e-25 -8.16740677e-15
  -3.39311904e-19]
 [ 1.38134859e-16  5.65218445e-13 -3.24202934e-18 -2.54109884e-21
  -1.38777878e-17 -1.18907213e-24  2.27167040e-16  2.07737403e-08
  -2.79434665e-20]
 [ 2.03904602e-18  6.61744490e-24 -2.75620286e-17  2.03877739e-08
   1.53592649e-17 -8.78910446e-14 -1.08451838e-19 -2.10911204e-19
  -2.16837840e-19]]


In [23]:
for n in range(N-1):
    vars = np.concatenate((q_o_sol[:, n+1], v_o_sol[:, n+1], vdot_o_sol[:, n], 
                           lambd_z_sol[:, n], 
                            lambd_x_pos_sol[:, n],
                            lambd_x_neg_sol[:, n]
                          ))
    print("manipulator_equations:", autoDiffToValueMatrix(manipulator_equations(vars=vars, context_index=n)))

manipulator_equations: [[-3.33066907e-16]
 [ 0.00000000e+00]
 [-3.36923603e-17]]
manipulator_equations: [[ 0.00000000e+00]
 [-1.11022302e-16]
 [-6.54236049e-14]]
manipulator_equations: [[-2.92734587e-18]
 [ 0.00000000e+00]
 [ 2.35665742e-18]]
manipulator_equations: [[0.00000000e+00]
 [1.11022302e-16]
 [9.39147780e-19]]
manipulator_equations: [[5.27327641e-24]
 [0.00000000e+00]
 [4.18673267e-19]]
manipulator_equations: [[ 8.32027374e-26]
 [ 1.11022302e-16]
 [-2.26216793e-18]]
manipulator_equations: [[-7.69928243e-27]
 [ 0.00000000e+00]
 [ 1.52599942e-17]]
manipulator_equations: [[-1.08420217e-19]
 [ 0.00000000e+00]
 [ 8.53021646e-18]]
manipulator_equations: [[-1.33672387e-21]
 [ 1.11022302e-16]
 [-4.72369430e-18]]


In [24]:
# with open(tmpfolder + 'test_only_box_passive_scenegraph_sliding_friction/sol.pkl', 'wb') as file:
#     pickle.dump( [q_o_sol, v_o_sol, vdot_o_sol, f_list_sol, alpha_beta_list_sol, 
#                   f_static_list_sol, f_sliding_list_sol, sliding_c_list_sol], file )

In [25]:
print(gravity)

[ 0.    0.   -9.81]
